In [ ]:
import copy
import time
import cloudpickle
import pandas as pd

from agent import policy
from hospital.people import Patient

Greedy optimisation finds the best allocation currently available given the state of the hospital and attributes of the patient. Consider the following example:
<ul>
<li>A female adult medical patient has arrived at the hospital and there are only two available beds. The first is within a bed bay of the male medical ward, the second is within a bed bay of a female surgical ward.
<li>According to user research the penalty for assigning a patient to ward with incorrect sex is 10 whilst the penalty for assigning a medical patient to a surgical ward is 3. 
<li>The greedy algorithm would choose the second bed, finding the solution that incurs the lowest penalty.
</ul>

In a more complicated scenario, where multiple penalties of varying cost may apply, it will optimise for the lowest aggregated penalty, which is a balance between the number of constraints broken and the magnitude of their costs i.e., breaking 1 constraint at a cost of 10 is worse than breaking 2 with a total cost of 7. 

This approach is called ‘greedy’ as it performs a brute force search across all available options. This may be slow if the size of your search space (e.g. number of possible choices) is very large.


In this notebook we demonstrate how to run the greedy allocation agent on the hospital that was created in notebook 1.


# 1. Load the Hospital Environment

We load the saved hospital environment and initialise it with a random set of patients at an occupancy of 90%. After populating the hospital you can see that there are patients attached to 90% of the beds, and also use the hospital functions to get a list of empty bed, understand the current penalty associated with the hospital (N.B. this will be very high as we randomly initialised the hospital).

In [ ]:
with open("../data/hospital.pkl", "rb") as f:
        h = cloudpickle.load(f)

In [ ]:
policy.populate_hospital(h, occupancy=0.9)
h.render()

In [ ]:
# function returns a generator so wrap in list to see all empty beds
len(list(h.get_empty_beds()))

In [ ]:
start_penalty = h.eval_restrictions()["score"]
print(f"Start penalty of the hospital: {start_penalty}")

# 2. Create a patient and Allocate

we can create a patient to allocation as seen in notebook 1 and use the `greedy_suggestions` function to return the top N bed suggestions for this patient, given the current state of the hospital. The suggestions are returned as a distionary with the bed names as keys and a dictionary of penalties and violated restrictions as values. Below we unpack this into a pandas DataFrame for easy comparison. 

The greedy allocation agent is plugged into the UI, please refer to the UI to see how this type of tool may be utilised by end users. 

In [ ]:
patient = Patient(
    name="patient",
    sex="female",
    department="medicine",
    specialty="general",
    is_immunosupressed=True,
)

In [ ]:
numer_of_suggestions = 5
start_time = time.time()
suggestions = policy.greedy_suggestions(h, patient, numer_of_suggestions)
elapsed = time.time() - start_time

In [ ]:
print(f"Time take to compute top {numer_of_suggestions} greedy suggestions: {elapsed}")

df = pd.DataFrame(suggestions).T
df